In [ ]:
#GP Regression

#AT as the only input
X1 = scale(np.array(df.drop(['PE','V','AP','RH'],1)))

X_train, X_test, y_train, y_test = train_test_split(X1,y,test_size = .001,random_state = 42)

#Preprocess data for Modeling
y_train.reshape(y_train[0],1)
model_input = shared(X_train)
model_output= shared(y_train)

print(X_test.shape,y_test.shape)


def kernel(a,b):
    sqdist = np.sum(a**2,1).reshape(-1,1)+np.sum(b**2,1)-2*np.dot(a,b.T)
    return np.exp(-5* sqdist)

n = X_test.shape[0]
k = kernel(X_train,y_train)
k_noise = 1e-6*np.eye(n)
L = np.linalg.cholesky(k+k_noise)

f_prior = np.dot(L,np.random.normal(size=(n,10)))

alpha = np.linalg.solve(L.T, np.linalg.solve(L, f_prior))
#post_mean = np.dot(K_s.T.eval(), alpha)
noise = 0.1


In [ ]:
with pm.Model() as model:
    # The actual distribution of f_sample doesn't matter as long as the shape is right since it's only used
    # as a dummy variable for slice sampling with the given prior
    f_sample = pm.Flat('f_sample', shape=(n, ))

    # Likelihood
    y = pm.MvNormal('y', observed=f_prior, mu=f_sample, cov=noise * T.eye(n), shape=n)


    # Interpolate function values using noisy covariance matrix
    L = T.slinalg.cholesky(k_noise)
    f_pred = pm.Deterministic('f_pred', T.dot(K_s.T, T.slinalg.solve(L.T, T.slinalg.solve(L, f_sample))))


    # Use elliptical slice sampling
    ess_step = pm.EllipticalSlice(vars=[f_sample], prior_cov=K_stable)
    trace = pm.sample(5000, start=model.test_point, step=[ess_step], progressbar=False, random_seed=1)

In [ ]:
np.random.seed(1)

# Number of training points
n = 30
X0 = np.sort(3 * np.random.rand(n))[:, None]


# Number of points at which to interpolate
m = 100
X = np.linspace(0, 3, m)[:, None]

# Covariance kernel parameters
noise = 0.1
lengthscale = 0.3
f_scale = 1

cov = f_scale * pm.gp.cov.ExpQuad(1, lengthscale)
K = cov(X_test)
k_ = cov(X_test,X_train)
K_noise = K + noise * T.eye(n)

# Add very slight perturbation to the covariance matrix diagonal to improve numerical stability
K_stable = K + 1e-12 * T.eye(n)
#print(K_noise.eval())


# Observed data
#f = np.random.multivariate_normal(mean=np.zeros(n), cov=K_noise.eval())

In [ ]:
np.random.seed(1)

# Number of training points
n = 30
X0 = np.sort(3 * np.random.rand(n))[:, None]


# Number of points at which to interpolate
m = 100
X = np.linspace(0, 3, m)[:, None]

# Covariance kernel parameters
noise = 0.1
lengthscale = 0.3
f_scale = 1

cov = f_scale * pm.gp.cov.ExpQuad(1, lengthscale)
K = cov(X0)
K_s = cov(X0, X)
K_noise = K + noise * T.eye(n)

# Add very slight perturbation to the covariance matrix diagonal to improve numerical stability
K_stable = K + 1e-12 * T.eye(n)

# Observed data
f = np.random.multivariate_normal(mean=np.zeros(n), cov=K_noise.eval())

print(X)
#print(K_noise)
#print(K_noise.eval())